In [1]:
# Changing the working directory
import os
os.chdir('../')

import gspread
from utils import *
from skydb.sheets import updateSpreadsheet, createSpreadsheet

In [2]:
records = getOfficialRecords()
grades = records['grades']
gpas = records['gpas']
students = getStudents().drop(['birth_date'], axis=1)
psat_scores = getPsatScores()

In [3]:
""" PSAT Scores """
psat_students = (
    students
    .query("""grade_level in [9,10,11]""")
    .drop('counselor', axis=1)
    .rename(columns={'id':'user_id'})
    .astype({'user_id':'int64', 'student_id':'float', 'grade_level':'int64'})
    .merge(psat_scores,
                'left',
                on='user_id'
    )
)

In [4]:
""" Cumulative GPA """
gpa_students = (
    psat_students
    .merge(gpas,
           'inner',
           on='user_id'
    )
)

In [5]:
""" Cleaning Science/Math Grades """
raw_science_grades = grades.loc[grades.transcript_category.str.contains('Science|Mathematics')].copy()
for index, row in raw_science_grades.iterrows():
    if row['school_year'] == '2021 - 2022' and row['term'] == 'Fall':
        raw_science_grades.loc[index, 'official_grade'] = row['grade']
    elif row['term'] == 'Spring':
        raw_science_grades.loc[index, 'official_grade'] = row['grade']
    else:
        raw_science_grades.loc[index, 'official_grade'] = None
        
science_grades = raw_science_grades[~raw_science_grades.official_grade.isna()].copy().drop('grade', axis=1)

""" Adding label for math/science classes"""
science_grades['label'] = (
    science_grades['transcript_category'] +
    (
        science_grades
        .groupby(['user_id', 'transcript_category'])
        .cumcount()
    ).astype('str')
)

science_grades.label = science_grades.label.str.replace(' ', '')

In [6]:
""" Spread Data """
clean_science_students = (
    science_grades
    .groupby(['user_id', 'label'])
    [['official_grade', 'course_title']]
    .first()
    .unstack()
    .copy()
)

""" Cleaning Column Names """
clean_science_students.columns = (
clean_science_students.columns.get_level_values(1) + " "+ clean_science_students.columns.get_level_values(0)
)

clean_science_students = clean_science_students[[
    'Mathematics0 course_title', 'Mathematics0 official_grade',
    'Science0 course_title', 'Science0 official_grade',
    'Mathematics1 course_title', 'Mathematics1 official_grade',
    'Science1 course_title', 'Science1 official_grade',
    'Mathematics2 course_title', 'Mathematics2 official_grade',
    'Science2 course_title', 'Science2 official_grade',
    'Mathematics3 course_title', 'Mathematics3 official_grade',
    'Science3 course_title', 'Science3 official_grade',
    'Science4 course_title', 'Science4 official_grade',
    'Science5 course_title', 'Science5 official_grade',
    'Science6 course_title', 'Science6 official_grade',
]].reset_index()


In [7]:
""" Joining with student Data """
science_data = (
    gpa_students
    .merge(clean_science_students,
           'inner',
           on='user_id'
    )
    .sort_values(['last_name', 'first_name'])
    .drop(['user_id', 'student_id', 'first_name'], axis=1)
)

In [8]:
# updateSpreadsheet(science_data.fillna(''), 
#                   sheet_id='1yf0CHCaG39S7gL65YOOVrPNgIqVyluC4fT1frQkSGRo',
#                   styleClass=HysonFireStyle
#                  )

In [ ]:
# Tableau Data 
updateSpreadsheet(english_data.fillna(''), 
                  sheet_id='1jeG6mJAEG819Mf7jRne8TKZMSh2KDXxzVuC9Y1M',
                  styleClass=HysonFireStyle
                 )